In [ ]:
import os
from typing import TypedDict,List,Union
from langchain_core.messages import HumanMessage,AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph,START,END
from dotenv import load_dotenv

load_dotenv()

class AgentState(TypedDict):
    messages:List[Union[HumanMessage,AIMessage]]

llm=ChatOpenAI(model="gpt-4o")


def process(state:AgentState)->AgentState:
    response=llm.invoke(state["messages"])
    state["messages"].append(AIMessage(content=response.content))
    print(f"\nAI: {response.content}\n")
    return state 


graph=StateGraph(AgentState)
graph.add_node("process",process)
graph.add_edge(START,"process")
graph.add_edge("process",END)

agent=graph.compile()

conversation_history=[]

user_input=input("You: ")
while user_input !="exit":
    conversation_history.append(HumanMessage(content=user_input))
    result=agent.invoke({"messages":conversation_history})
    conversation_history=result["messages"]
    user_input=input("You: ")

with open("conversation.txt","w") as file:
    file.write("Your Conversation History:\n")

    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n")
    file.write("\nEnd of Conversation\n")

print("Conversation history saved to conversation.txt")